In [0]:
from silver.Transform_Functions.Trim import func_trim_col
from pyspark.sql import functions as F
from pyspark.sql.functions import trim,col

# READING FROM BRONZE TABLE

In [0]:
df = spark.table("workspace.bronze.crm_prd_info_raw")

# Data Transformation

## Trim whitespaces

In [0]:
df = func_trim_col(df)

## Parse Category ID and Product Key from prd_key

In [0]:
df = df.withColumn("Cat_ID", F.concat_ws("_",
                                          F.split(F.col("prd_key"), "-").getItem(0),
                                          F.split(F.col("prd_key"), "-").getItem(1)))

In [0]:
df = df.withColumn("prd_key", F.regexp_replace(F.col("prd_key"), r"^[^-]+-[^-]+-", ""))

## Handle Null Value - PRD_COST

In [0]:
df = df.withColumn("prd_cost", F.coalesce(col("prd_cost"), F.lit(0)))

In [0]:
df.display()

## Normalize Abbreviation

In [0]:
df = (
    df.withColumn(
        "prd_line",
        F.when(F.upper(F.col("prd_line")) == "R", "Road")
        .when(F.upper(F.col("prd_line")) == "S", "Other Sales")
        .when(F.upper(F.col("prd_line")) == "M", "Mountain")
        .when(F.upper(F.col("prd_line")) == "T", "Touring")
        .otherwise("n/a")
    )
)

## Validate Data Type

In [0]:
df.printSchema()

## Renaming the columns

In [0]:
prd_table_header = {
    "prd_id": "product_id",
    "prd_key": "product_key",
    "prd_nm": "product_name",
    "prd_cost": "product_cost",
    "prd_line": "product_line",
    "prd_start_dt": "start_date",
    "prd_end_dt": "end_date",
    "Cat_ID": "category_id"}


In [0]:
for old_header, new_header in prd_table_header.items():
    df = df.withColumnRenamed(old_header, new_header)

# Write into Silver Schema

In [0]:
(
    df.write.mode("overwrite").format("delta").saveAsTable("silver.crm_products")
)


In [0]:
%sql
SELECT * FROM silver.crm_products
LIMIT 5;